In [ ]:
from utils import print_results, get_intents_selection, filter
from custom_embeddings import create_embed_f
from ADBThreshold import ADBThreshold
from ood_train import evaluate
from sentence_transformers import SentenceTransformer

import os, json, copy
import tensorflow_hub as hub
from statistics import mean

time_pretraining = None
accuracy_lst = []
f1_lst = []
time_train_lst = []
time_inference_lst = []
memory_lst = []
time_pretraining_lst = []

# 1 - LOAD DATASET
# ------------------------------------------------------------
dataset_name = 'clinc150-data_full'
dataset_path = os.path.join("datasets", "CLINC150", 'data_full.json')
# ------------------------------------------------------------

with open("./datasets/CLINC150/data_full.json") as f:
    old_dataset = json.load(f)


with open("results_2.txt", "w") as f:
    for KNOWN_RATIO, alpha in zip([0.25, 0.50, 0.75], [0.35, 0.4, 0.75]):
        for test_idx in [1, 2, 3, 4, 5, 6, 7, 8, 9]:
            
            time_pretraining = None
            accuracy_lst = []
            f1_lst = []
            time_train_lst = []
            time_inference_lst = []
            memory_lst = []
            time_pretraining_lst = []


            for r in range(10):
                model = ADBThreshold(alpha=alpha)
                model_name = type(model).__name__
                
                if test_idx == 1 or test_idx == 4 or test_idx == 5 :
                    embed_f = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
                    emb_name = 'use_dan'

                if test_idx == 2 or test_idx == 6 or test_idx == 7:
                    embed_f = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
                    emb_name = 'use_tran'

                if test_idx == 3 or test_idx == 8 or test_idx == 9:
                    embed_f = SentenceTransformer('stsb-roberta-base').encode
                    emb_name = 'sbert'

                dataset = copy.deepcopy(old_dataset)

                num_classes = len(set([x[1] for x in dataset['train']]))
                num_sel_classes = int(KNOWN_RATIO * num_classes)
                selection = get_intents_selection(dataset['train'], num_intents=num_sel_classes)

                filt_train = filter(dataset['train'], selection, 'train')
                filt_test = filter(dataset['test'], selection, 'test')

                dataset['train'] = filt_train
                dataset['test'] = filt_test

                # 4B-2 - CHOOSE EMBEDDING (with pre-training)
                if test_idx == 4:
                    embed_f, time_pretraining = create_embed_f(embed_f, dataset, limit_num_sents=None, type='cosface')
                    emb_name = 'use_dan_cosface'

                if test_idx == 5:
                    embed_f, time_pretraining = create_embed_f(embed_f, dataset, limit_num_sents=None, type='triplet_loss')
                    emb_name = 'use_dan_triplet_loss'

                if test_idx == 6:
                    embed_f, time_pretraining = create_embed_f(embed_f, dataset, limit_num_sents=None, type='cosface')
                    emb_name = 'use_tran_cosface'

                if test_idx == 7:
                    embed_f, time_pretraining = create_embed_f(embed_f, dataset, limit_num_sents=None, type='triplet_loss')
                    emb_name = 'use_tran_triplet_loss'

                if test_idx == 8:
                    embed_f, time_pretraining = create_embed_f(embed_f, dataset, limit_num_sents=None, type='cosface')
                    emb_name = 'sbert_cosface'

                if test_idx == 9:
                    embed_f, time_pretraining = create_embed_f(embed_f, dataset, limit_num_sents=None, type='triplet_loss')
                    emb_name = 'sbert_triplet_loss'

                results_dct = evaluate(dataset, model, model_name, embed_f, limit_num_sents=None)

                accuracy_lst.append(results_dct['accuracy'])
                f1_lst.append(results_dct['f1'])
                time_train_lst.append(results_dct['time_train'])
                time_inference_lst.append(results_dct['time_inference'])
                memory_lst.append(results_dct['memory'])

                if time_pretraining is not None:
                    time_pretraining_lst.append(time_pretraining)

                print_results(dataset_name, model_name, emb_name, results_dct)

            print(f"{test_idx}")
            print(f'Dataset: {dataset_name}, known ratio: {KNOWN_RATIO}')
            print(
                f'accuracy: {round(mean(accuracy_lst), 1)},'
                f' f1: {round(mean(f1_lst), 1)},'
                f' time_train: {round(mean(time_train_lst), 1)},'
                f' time_inference: {round(mean(time_inference_lst), 1)},'
                f' memory: {round(mean(memory_lst), 1)},'
                f' time_pretraining: {round(mean(time_pretraining_lst), 1) if len(time_pretraining_lst) != 0 else 0}')
            
            f.write(
                f'Type={emb_name}; known_ratio={KNOWN_RATIO};'
                f' test_idx={test_idx};'
                f' accuracy={round(mean(accuracy_lst), 1)};'
                f' f1={round(mean(f1_lst), 1)};'
                f' time_train={round(mean(time_train_lst), 1)};'
                f' time_inference={round(mean(time_inference_lst), 1)};'
                f' memory={round(mean(memory_lst), 1)};'
                f' time_pretraining={round(mean(time_pretraining_lst), 1) if len(time_pretraining_lst) != 0 else 0}')
            f.write("\n")


INFO:absl:Using /tmp/tfhub_modules to cache modules.


dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 89.2, 'f1': 80.8, 'time_train': 1.9, 'time_inference': 1.5, 'memory': 1754.0}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 88.7, 'f1': 78.8, 'time_train': 1.8, 'time_inference': 1.5, 'memory': 2143.9}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 89.0, 'f1': 77.7, 'time_train': 1.9, 'time_inference': 1.6, 'memory': 2226.0}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 89.5, 'f1': 81.3, 'time_train': 1.8, 'time_inference': 1.5, 'memory': 2475.5}



...

dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 88.4, 'f1': 79.6, 'time_train': 1.9, 'time_inference': 1.5, 'memory': 2538.1}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 87.4, 'f1': 77.8, 'time_train': 1.9, 'time_inference': 1.5, 'memory': 2708.5}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 87.6, 'f1': 77.2, 'time_train': 1.9, 'time_inference': 1.5, 'memory': 2600.1}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 87.4, 'f1': 77.8, 'time_train': 1.9, 'time_inference': 1.5, 'memory': 2711.5}



dataset_name: clinc150-data_full, model_name: ADBThreshold, embedding_name: use_dan -- {'accuracy': 86.3, 'f1': 74.9, 'time_train': 1.9, 'time_inference': 1.5, 'memory': 2775.9}

1
Dataset: clinc150-data_full, known ratio: 0.25
accuracy: 88.1, f1: 78.4, time_train: 1.9, time_inference: 1.5, memory: 2458.5, time_pretraining: 0


In [1]:
!pip install -U -q sentence-transformers